<a href="https://colab.research.google.com/github/Manjunath727/DLwithTF/blob/master/CNN_Using_TF/W2/ConvNet_with_DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Horse or Human with Augmentation

### Download data, setup dirs

Following code download the zip file, extracts and sets up training and validation directories for horses and humans


In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip
  
  
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
with zipfile.ZipFile(local_zip) as zip_ref:
  zip_ref.extractall('/tmp/horse-or-human')
  zip_ref.close()

local_zip = '/tmp/validation-horse-or-human.zip'
with zipfile.ZipFile(local_zip) as zip_ref:
  zip_ref.extractall('/tmp/validation-horse-or-human')
  zip_ref.close()

# Directory with training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses/')

# Directory with training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with validation horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses/')

# Directory with validation human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans/')

--2019-07-06 14:04:09--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  94.5MB/s    in 1.5s    

2019-07-06 14:04:11 (94.5 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2019-07-06 14:04:12--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/

### Build a model
First we define our neural network model. As usual we use convolutional layers and flatten the final result to feed it to a dense network layer. Finally we add the dense connected layers.

Since this is a binary classification problem, we use sigmoid function

In [0]:
import tensorflow as tf

In [3]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 color
    # bytes
    # First convolution
    tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Second convolution
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Third convolution
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Fourth convolution
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Fifth convolution
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='sigmoid')
])

W0706 14:04:16.642902 140114716776320 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### **Compile the model**

We compile the model with loss functin set as 'binary_cross_entropy' and use RMSprop as the optimizer to set the learning rate

In [4]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['acc'])


W0706 14:04:16.999677 140114716776320 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Data Augmentation

`ImageDataGenerator` class of keras library provides methods to generate batches tensor data with real time data augmentation. The data will be looped over in batches.
These generators can then be used with Keras model methods that accept data generators as inputs: `fit_generator`, `evaluate_generator` and `predict_generator`


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# all images will be scaled by 1./255.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    '/tmp/horse-or-human',       # Source dir of the training images
    target_size=(300,300),
    batch_size=128,
    # Since we use binary cross_entropy loss, we need nbinary labels
    class_mode='binary'
)

# Flow training images in batches of 32 using validataion datagen generator
validation_generator = validation_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human', # Source dir of the validation images
    target_size=(300,300),
    batch_size = 32,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


### Train the model



In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=100,
    verbose=1,
    validation_data = validation_generator,
    validation_steps=8
)

Epoch 1/100
8/8 [==============================] - 107s 13s/step - loss: 0.6883 - acc: 0.5261 - val_loss: 0.6975 - val_acc: 0.5000
Epoch 2/100
8/8 [==============================] - 103s 13s/step - loss: 0.6734 - acc: 0.5662 - val_loss: 0.6613 - val_acc: 0.5039
Epoch 3/100
8/8 [==============================] - 103s 13s/step - loss: 0.6528 - acc: 0.5706 - val_loss: 0.6163 - val_acc: 0.8008
Epoch 4/100
8/8 [==============================] - 103s 13s/step - loss: 0.6451 - acc: 0.6730 - val_loss: 0.6338 - val_acc: 0.5391
Epoch 5/100
6/8 [=====================>........] - ETA: 25s - loss: 0.6030 - acc: 0.7318

### Evaluating loss and accuracy of model


In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()